# To do list:
- limit the getting of data using a date
- create different types of arcgis actions (publishing new service, overwritting a new service, append to a new service)

In [ ]:
import json
import requests
import re
from pprint import pprint
import pandas as pd

# Functions

In [ ]:
def executeRequest(url, headers, payload):
    
    try:
    ## Create empty object first
        r_json = {}
    
    ## cleaner way of using requests
        response = requests.get(url, headers=headers, data=payload)

    except:
        print("There was an problem in the request :(")
        return None

    ## always nice to print the url as a sanity check
    print(response.url)

    # if succesful, populate your response json
    if  response.status_code == 200:
            r_json = response.json()
    else:
        print(f'Failed to get data {response.status_code}, {response.json()}')
    
    return r_json

In [ ]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
    except:
        print("There was a problem with the string.")
    return reg_exp

In [ ]:
def searchContactListsID(r_json, reg_exp):
    try:
        l_json = r_json.get("lists", [])
        assert type(l_json) is list and len(l_json) is not 0,  "Error with l_json"
        sel_contact_dict = {
        d['name']: d.get('list_id', '')
        for d in l_json
        if re.search(reg_exp, d['name']) != None
        }        
    except:
        print("There was a problem with the structure of the json")
    return sel_contact_dict

In [ ]:
def requestURLbyListID(id_contact_list):
    try:
        url = f"https://api.cc.email/v3/contacts?lists={id_contact_list}&include=street_addresses&limit=500&include_count=false"
    except:
        print("There was a problem with the id.")
    return url

In [ ]:
def getContactsLocation(r_json):
    try:
        contacts_list = []
        l_json = r_json.get("contacts", [])
        for d in l_json:
            street_address = d.get('street_addresses', [{}])#[0]
            if street_address:
                street_address = street_address[0]
                postal_code = street_address.get("postal_code", None)
                country = street_address.get("country", None)
                if postal_code and country:
                    contact_dict = {
                        'contact_id': d.get('contact_id', ''), ## Need a fallback for contact_id? No, there is always a contact_id
                        'postal_code': postal_code,
                        'country': country
                    }
                    contacts_list.append(contact_dict)  
        df = pd.DataFrame(contacts_list)
    except:
        print("There was a problem with the structure of the json")
    return df

In [ ]:
def missLocation(r_json, df):
    try: 
        l_json = r_json.get("contacts", [])
        all_contacts_list = []
        for d in l_json:
            all_contacts_list.append(d.get('contact_id', ''))
        original_set = set(all_contacts_list)
        located_set = set(df.contact_id)
        contact_diff = original_set.difference(located_set)
    except:
        print("There was a problem with the structure of the json")
    return contact_diff

In [ ]:
def writeLocationCsv(df, csvName):
    try:
        csv_file = f'./{csvName}.csv'
        df.to_csv(csv_file, index=False)
        print(f"{csv_file} written")
    except:
        print("The csv hasn't been written")

# Environmental variables

In [6]:
env_path = ".env"

In [7]:
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [ ]:
api_key = env['cc_api_key']
token = env['cc_token']

To get the token put this in the web browser: https://api.cc.email/v3/idfed?client_id={api_key}&redirect_uri=https://localhost&response_type=token&scope=contact_data

In [ ]:
f"https://api.cc.email/v3/idfed?client_id={api_key}&redirect_uri=https://localhost&response_type=token&scope=contact_data"

and then update the `.env` file. Is there a way of getting the url where this get call takes?

In [3]:
contact_lists_of_interest = ["Educator Ambassadors"]

In [19]:
testing_val = contact_lists_of_interest[0]

# Getting the data from Constant Contact

In [ ]:
headers = {
  'Authorization': f'Bearer {token}'
}
payload = {}

In [ ]:
url = "https://api.cc.email/v3/contact_lists?include_count=false"

In [ ]:
r_contact_lists = executeRequest(url, headers, payload)

In [ ]:
for list_element in contact_lists_of_interest:
    cl_to_search = create_strict_reg_exp(list_element)
    id_dict = searchContactListsID(r_json = r_contact_lists, to_search = cl_to_search)
    url_contacts = requestURLbyListID(id_dict[list_element])
    r_contacts = executeRequest(url_contacts, headers, payload)
    contacts_location_df = getContactsLocation(r_contacts)
    contacts_without_location = missLocation(r_contacts,contacts_location_df) #this should be written in some kind of log
    writeLocationCsv(contacts_location_df, list_element)

# Once the csv is ready it can be published in arcgis online via the arcgis api
Here there can be different cases:
- publish a new service, if the list is a new one
- fully overwrite a service
- append data to a service

In [4]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

In [87]:
def publishCSVasFS(csvName, sharing = "everyone", aol_folder_name = "constant_contact", aol_username = env['aol_username'], aol_password = env['aol_key']):
    try:
        gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
        #check that the feature service name doesn't exist already
        # is_service_name_available(service_name, service_type)
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            csv_file = f'./{csvName}.csv'
            csv_item = gis.content.add({}, csv_file)
            csv_lyr = csv_item.publish(None,  { 'CountryCode' : 'country',
                                            'Postal' : 'postal_code'} )
            print(f"Service {csvName} has been published")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
            #sharing
            if sharing == "everyone":
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")
            else:
                print(f"not public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            try:
                csv_lyr.delete()
                print("The service has not been published.")
            except:
                print("The detele protection is activated.")
            #mark deprecated
            # it is possible to check the status with csv_item.content_status
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
        
    except:
        print("The csv hasn't been published")

In [88]:
publishCSVasFS(csvName = testing_val)

Service name Educator Ambassadors is available
Service Educator Ambassadors has been published
Moving service Educator Ambassadors to constant_contact in ArcGIS Online...
Service Educator Ambassadors has been moved to constant_contact in ArcGIS Online
shared with everyone
shared with org
not shared with groups


Unable to delete item d3afbe6805604fa78784cddecbd0404f. Delete protection is turned on.


The detele protection is activated.


To overwrite follow [this notebook from ESRI](https://developers.arcgis.com/python/sample-notebooks/overwriting-feature-layers/). 

In [ ]:
def overwriteFSwithCSV(csvName, aol_folder_name = "constant_contact"):
    

To append follow [this notebook from ESRI](https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/).

In [ ]:
def appendCSVtoFS(csvName, aol_folder_name = "constant_contact"): 
    

In [ ]:
def csvToArcgis(csvName, action, aol_folder_name = "constant_contact"):
    if action == "publish":
        publishCSVasFS(csvName)
    if action == "overwrite":
        overwriteFSwithCSV()
    if action == "append":
        appendCSVtoFS()

In [9]:
aol_password = env['aol_key']
aol_username = env['aol_username']

In [10]:
#getting into the GIS
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [ ]:
aol_folder_name = "constant_contact"

In [ ]:
for list_element in contact_lists_of_interest:
    csv_file = f'./{list_element}.csv'
    publishCSVasFS(csv_file)

In [97]:
test_item = gis.content.search(testing_val, item_type = "Feature Layer")[0]

In [ ]:
new_folder_details = gis.content.create_folder(aol_folder_name)
csv_item.move(new_folder_details)
csv_lyr.move(new_folder_details)
csv_lyr.share(everyone = False)
#print(csv_lyr.ownerFolder)